Start by downloading and opening a real image from PTF (https://ui.adsabs.harvard.edu/abs/2009PASP..121.1395L/abstract)

In [ ]:
import os
import subprocess

data_path = 'data'
if not os.path.isdir(im_path):
    os.mkdir(im_path)

im_link = (
    'https://www.dropbox.com/scl/fi/g267d8kqcgytet5rwa3wa/'
    'PTF_20100216_075004_11_R_Diff_B74YFO_u-r4yk3x.image.fits?'
    'rlkey=9jhf3ox8y90wo4bc9g0b710zh&st=678qblv0&dl=1'
)
im_filename = 'PTF_20100216_075004_11_R_Diff_B74YFO_u-r4yk3x.image.fits'
im_filepath = os.path.join(data_path, im_filename)

if not os.path.isfile(im_filepath):
    subprocess.run(['wget', im_link, '-O', im_filepath])

fl_link = (
    'https://www.dropbox.com/scl/fi/izw2nrdwji3hn0608izbt/'
    'PTF_20100216_075004_11_R_Diff_B74YFO_u-r4yk3x.flags.fits?'
    'rlkey=foe3ymm5va6fq9wwq8jr1od6a&st=mo8jf74y&dl=0'
)
fl_filename = 'PTF_20100216_075004_11_R_Diff_B74YFO_u-r4yk3x.flags.fits'
fl_filepath = os.path.join(data_path, fl_filename)

if not os.path.isfile(fl_filepath):
    subprocess.run(['wget', fl_link, '-O', fl_filepath])

In [ ]:
import os
from astropy.io import fits

with fits.open(im_filepath) as hdu:
    im = hdu[0].data

with fits.open(fl_filepath) as hdu:
    flags = hdu[0].data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

im_path = 'data'
im_filename = 'PTF_20100216_075004_11_R_Diff_B74YFO_u-r4yk3x.image.fits'
with fits.open(os.path.join(im_path, im_filename)) as hdu:
    im = hdu[0].data

flags_filename = 'PTF_20100216_075004_11_R_Diff_B74YFO_u-r4yk3x.flags.fits'
with fits.open(os.path.join(im_path, flags_filename)) as hdu:
    flags = hdu[0].data

imnan = im.copy()
imnan[flags>0] = np.nan;
print(im.shape)
plt.figure()
plt.imshow(imnan, vmin=-5, vmax=5)

To make this run a lot faster we can cut a 1024x1024 square around a place we already see has a streak. 
The finder can do something similar by making sections internally using `use_sections`. This will just speed things up. 

In [ ]:
section = imnan[2048:3072, 1024:2048]
plt.figure()
plt.imshow(section, vmin=-5, vmax=5)

In [ ]:
import time
from src.finder import Finder
t0 = time.perf_counter()
f = Finder(verbosity=10, use_show=False, threshold=7.5, use_exclude=False)
f.input(section)
t1 = time.perf_counter() - t0
print(f"Found {len(f.streaks)} streaks in {t1:.2f}s")

The first streak that was detected is the very bright and obvious satellite trail on the bottom of the section. 
This is an easy target for streak detection, but much fainter streaks can also be found. 
Note that because of the way the image is sliced into powers of 2, the streak tracks do not match 
the streak start/end points exactly. Additional fitting of the streak cutout (using the initial fit) is recommended. 

In [ ]:
plt.figure(); f.streaks[0].show(vmin=-3, vmax=5)

The remaining streaks that were detected are false positives. 
Mostly they are due to the weird "ghost" artefact on the left-hand side of the image. 
There's also a long streak on the right-hand side that seems to be just triggering on 
noise and maybe the bright edges of the image. 
Real images must apply additional filtering of the streaks that were detected. 
The FRT algorithm can detect faint streaks, but has no mechanism for removing splotches 
or even just random false alarms. This is especially true if the background is not uniform
or if the noise RMS is higher than what was given to the `Finder`. 

In [ ]:
plt.figure() 
for i in range(1, len(f.streaks)):
    f.streaks[i].show(vmin=-3, vmax=5)
    

Try to add lots of white noise to this image. The bright satellite trail is now much fainter.  

In [ ]:
sigma = 8
section2 = section + np.random.normal(0, sigma, size=section.shape)
sigma = np.sqrt(sigma ** 2 + np.nanvar(section))  # make sure to account for the real image variance
plt.figure()
plt.imshow(section2, vmin=-sigma, vmax=sigma)

In [ ]:
f.input(section2, variance=sigma**2)
len(f.streaks)

The streak is still detectable, only just above the threshold of 7.5 sigma

In [ ]:
plt.figure(); f.streaks[0].show(vmin=-sigma, vmax=sigma)
f.streaks[0].snr